In [2]:
import pandas as pd
import numpy as np
import warnings
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
warnings.simplefilter(action='ignore')

In [3]:
# 1. 페이지 접근
# 2. 서울 선택
# 3. 전체 선택
# 4. 검색 결과 리스트 가져오기(beautifulsoup)
# 5. 데이터 수집(매장 이름, 주소, 위도, 경도)
# 6. 전체 데이터 수집

In [5]:
# 1. 스타벅스 페이지 접근

url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver = webdriver.Chrome()
driver.get(url)

In [7]:
# 팝업창 닫기
# 팝업창이 없을 수도 있습니다.
driver.find_element(By.CSS_SELECTOR, '.holiday_notice_close a').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".holiday_notice_close a"}
  (Session info: chrome=115.0.5790.102); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000103193f48 chromedriver + 4226888
1   chromedriver                        0x000000010318c4f4 chromedriver + 4195572
2   chromedriver                        0x0000000102dd0d68 chromedriver + 281960
3   chromedriver                        0x0000000102e0bfc8 chromedriver + 524232
4   chromedriver                        0x0000000102e42c58 chromedriver + 748632
5   chromedriver                        0x0000000102dfff1c chromedriver + 474908
6   chromedriver                        0x0000000102e00ef4 chromedriver + 478964
7   chromedriver                        0x000000010315559c chromedriver + 3970460
8   chromedr

In [8]:
# 2. 서울 선택
driver.find_element(By.CSS_SELECTOR, '.sido_arae_box li a').click()

In [9]:
# 3. 서울 전체 선택
driver.find_element(By.CSS_SELECTOR, '.gugun_arae_box li a').click()

In [11]:
# 4. 전체 리스트 가져오기
seoul_list = driver.find_elements(By.CSS_SELECTOR, '#mCSB_3_container ul li')
len(seoul_list)

601

In [12]:
# beautifulsoup
html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

In [13]:
# 5. 매장이름, 주소, 구, 위도, 경도
seoul_list = dom.select('#mCSB_3_container ul li')
title = seoul_list[0]['data-name']
address = seoul_list[10].select_one('p').text[:-9]
gu_name = address.split()[1]
lat = seoul_list[0]['data-lat']
lng = seoul_list[0]['data-long']

len(seoul_list), title, address, gu_name, lat, lng

(601, '역삼아레나빌딩', '서울특별시 강남구 광평로 281 (수서동)', '강남구', '37.501087', '127.043069')

In [14]:
# 6. 전체 데이터 수집

datas = []

for content in tqdm_notebook(seoul_list):
    try:
        title = content['data-name']
        address = content.select_one('p').text[:-9]
        lat = content['data-lat']
        lng = content['data-long']
        gu_name = address.split()[1]
        datas.append({
            'title': title,
            'address': address,
            'gu': gu_name,
            'lat': lat,
            'lng': lng
        })
    except Exception as e:
        # 에러가 발생할 경우 에러메시지 출력 후 드라이버 종료
        print(e)
        driver.close()

driver.close()
df = pd.DataFrame(datas)
df.tail()

  0%|          | 0/601 [00:00<?, ?it/s]

,title,address,gu,lat,lng
596,사가정역,서울특별시 중랑구 면목로 310,중랑구,37.579594,127.087966
597,상봉역,서울특별시 중랑구 망우로 307 (상봉동),중랑구,37.59689,127.08647
598,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구,37.615368,127.076633
599,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구,37.6066536267232,127.106359790053
600,중화역,서울특별시 중랑구 봉화산로 35,중랑구,37.60170912407773,127.07841136432036


In [15]:
# 상단 데이터 확인
df.head(1)

,title,address,gu,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),강남구,37.501087,127.043069


In [16]:
# 하단 데이터 확인
df.tail(1)

,title,address,gu,lat,lng
600,중화역,서울특별시 중랑구 봉화산로 35,중랑구,37.60170912407773,127.07841136432036


In [17]:
# 데이터 information 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    601 non-null    object
 1   address  601 non-null    object
 2   gu       601 non-null    object
 3   lat      601 non-null    object
 4   lng      601 non-null    object
dtypes: object(5)
memory usage: 23.6+ KB


In [18]:
# csv 파일 저장
df.to_csv('./starbucks.csv', encoding='utf-8')

In [19]:
# 파일 읽기
starbucks_df = pd.read_csv('./starbucks.csv', index_col=0)
starbucks_df

,title,address,gu,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),강남구,37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),강남구,37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),강남구,37.513931,127.020606
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),강남구,37.499517,127.031495
4,대치재경빌딩R,서울특별시 강남구 남부순환로 2947 (대치동),강남구,37.494668,127.062583
...,...,...,...,...,...
596,사가정역,서울특별시 중랑구 면목로 310,중랑구,37.579594,127.087966
597,상봉역,서울특별시 중랑구 망우로 307 (상봉동),중랑구,37.596890,127.086470
598,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구,37.615368,127.076633
599,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구,37.606654,127.106360
